# Example usage of Adaptive Attention Generation for Indonesian Image Captioning
In this notebook, we will explore how to install and predict a single image

# Installation

In [ ]:
!git clone https://github.com/share424/Adaptive-Attention-Generation-for-Indonesian-Image-Captioning.git

In [ ]:
!pip install Adaptive-Attention-Generation-for-Indonesian-Image-Captioning

# Download pretrained models

In [ ]:
!wget https://github.com/share424/Adaptive-Attention-Generation-for-Indonesian-Image-Captioning/releases/download/Weights/weights-resnet101.zip

!unzip weights-resnet101.zip

# Download the image